# Credentials

Please DO NOT share this notebook with your credentials!

In [ ]:
D4S_USERNAME = "" # @param {type:"string"}
D4S_PASSWORD = "" # @param {type:"string"}

# Configuration

In [ ]:
location = "United States" # @param {type:"string"}
language = "en" # @param {type:"string"}

# Keywords

One keyword per line, inside a multi-line string (triple quotes).

In [ ]:
keywords_text = """
avocado recipes
avocado toast recipes
"""

## We separate our keyword list from text into an unique Python list

In [ ]:
keywords = keywords_text.split("\n")
keywords = [kw for kw in keywords if kw.strip() != ""]
keywords = list(set(keywords))
keywords

['avocado recipes', 'avocado toast recipes']

# Code

## Boilerplate code for D4S

In [ ]:
from http.client import HTTPSConnection
from base64 import b64encode
from json import loads, dumps, dump

class RestClient:
    domain = "api.dataforseo.com"

    def __init__(self, username, password):
        self.username = username
        self.password = password

    def request(self, path, method, data=None):
        connection = HTTPSConnection(self.domain)
        try:
            base64_bytes = b64encode(
                ("%s:%s" % (self.username, self.password)).encode("ascii")
                ).decode("ascii")
            headers = {'Authorization' : 'Basic %s' %  base64_bytes, 'Content-Encoding' : 'gzip'}
            connection.request(method, path, headers=headers, body=data)
            response = connection.getresponse()
            return loads(response.read().decode())
        finally:
            connection.close()

    def get(self, path):
        return self.request(path, 'GET')

    def post(self, path, data):
        if isinstance(data, str):
            data_str = data
        else:
            data_str = dumps(data)
        return self.request(path, 'POST', data_str)

## Get the current date

This will be used on our final Excel file.

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-03-29'

# Get all SERPs for our keywords

In [ ]:
client = RestClient(D4S_USERNAME, D4S_PASSWORD)

all_results = []

for keyword in keywords:  
  post_data = dict()

  post_data[len(post_data)] = dict(
      language_code=language,
      location_name=location,
      keyword=keyword
  )
      
  try:
    response = client.post("/v3/serp/google/organic/live/regular", post_data)
  except:
    response = None
    print(f'Error getting keyword {keyword} SERP')

  if response:    
    if response["status_code"] == 20000:
      data = response['tasks'][0]["result"][0]["items"]    
      
      print(f'Got SERP for "{keyword}""')

      for item in data:
        item["keyword"] = keyword
        item["date"] = today
        all_results.append(item)        
    else:
      print("error. Code: %d Message: %s" % (response["status_code"], response["status_message"]))  

Got SERP for "avocado recipes""
Got SERP for "avocado toast recipes""


In [ ]:
f'Got {len(all_results)} results for {len(keywords)} keywords'

'Got 194 results for 2 keywords'

## Lets take a look at the first keyword SERP

In [ ]:
import pandas as pd

df = None
if len(all_results) > 0:
  df = pd.DataFrame.from_dict(all_results)

In [ ]:
df.head()

,type,rank_group,rank_absolute,domain,title,description,url,breadcrumb,keyword,date
0,organic,1,3,www.loveandlemons.com,54 Avocado Recipes for Every Meal - Love and L...,Salads with Avocado. Broccoli Pesto Quinoa Sal...,https://www.loveandlemons.com/avocado-recipes/,www.loveandlemons.com › Recipes,avocado recipes,2021-03-29
1,organic,2,4,www.tasteofhome.com,50 Amazing Avocado Recipes You Need to Try | T...,"Sep 24, 2018 — 50 Amazing Avocado Recipes You ...",https://www.tasteofhome.com/collection/recipes...,www.tasteofhome.com › ... › Vegetables › Avocados,avocado recipes,2021-03-29
2,organic,3,5,www.delish.com,50+ Easy Avocado Recipes - Best Dishes With Av...,"Jun 24, 2020 — 51 Avocado Recipes So You Never...",https://www.delish.com/cooking/recipe-ideas/g2...,www.delish.com › cooking › recipe-ideas › thin...,avocado recipes,2021-03-29
3,organic,4,6,www.bonappetit.com,"56 Avocado Recipes, So You Can Eat As Much of ...","Feb 18, 2020 — 56 Avocado Recipes, So You Can ...",https://www.bonappetit.com/recipes/slideshow/4...,www.bonappetit.com › recipes › slideshow › 40-...,avocado recipes,2021-03-29
4,organic,5,7,www.thespruceeats.com,25 Ways to Use Avocado in Your Next Meal - The...,"Sep 13, 2020 — Avocado and Egg Toast · Buffalo...",https://www.thespruceeats.com/avocado-recipes-...,www.thespruceeats.com › avocado-recipes-4164909,avocado recipes,2021-03-29


In [ ]:
df.tail()

,type,rank_group,rank_absolute,domain,title,description,url,breadcrumb,keyword,date
189,organic,93,97,teabreakfast.com,20 Tasteful Avocado Toast Recipes for Healthy ...,"Feb 21, 2021 — It's a different way of enhanci...",https://teabreakfast.com/avocado-toast-recipe/,teabreakfast.com › Breakfast Recipes,avocado toast recipes,2021-03-29
190,organic,94,98,chickenofthesea.com,Tuna Avocado Toast with Tomato | Chicken of th...,Step 1. Toast the bread. Slice the avocado and...,https://chickenofthesea.com/recipes/tuna-avoca...,chickenofthesea.com › recipes › tuna-avocado-t...,avocado toast recipes,2021-03-29
191,organic,95,99,www.americastestkitchen.com,Kid-Friendly Avocado Toast with Fried Eggs | A...,Avocado toast is a healthy and simple snack th...,https://www.americastestkitchen.com/kids/recip...,www.americastestkitchen.com › kids › recipes ›...,avocado toast recipes,2021-03-29
192,organic,96,100,weelicious.com,Avocado Toast | Weelicious,"4 slices favorite bread, toasted · 1 small rip...",https://weelicious.com/avocado-toast-recipe/,weelicious.com › avocado-toast-recipe,avocado toast recipes,2021-03-29
193,organic,97,101,showmetheyummy.com,Avocado Toast - 15-Minute Breakfast - FOUR Ways!,"Sep 8, 2020 — Today's “ recipes ” for avocado ...",https://showmetheyummy.com/avocado-toast/,showmetheyummy.com › Recipes › Breakfast › Savory,avocado toast recipes,2021-03-29


## Download as an Excel file

In [ ]:
from google.colab import files

filename = 'serp-results.xlsx'
df.to_excel(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>